In [1]:
import sys
import os

# Source: https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__name__))))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

import category_encoders as ce

from lui.ecoc import SVMRandomInvariantsECOC
from lui.types import InvariantTypes
from utils import run_experiment

In [2]:
df = pd.read_csv("../data/yeast.data", header=None)
df

,0,1,2,3,4,5,6,7,8,9
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT
...,...,...,...,...,...,...,...,...,...,...
1479,YUR1_YEAST,0.81,0.62,0.43,0.17,0.5,0.0,0.53,0.22,ME2
1480,ZIP1_YEAST,0.47,0.43,0.61,0.40,0.5,0.0,0.48,0.47,NUC
1481,ZNRP_YEAST,0.67,0.57,0.36,0.19,0.5,0.0,0.56,0.22,ME2
1482,ZUO1_YEAST,0.43,0.40,0.60,0.16,0.5,0.0,0.53,0.39,NUC


In [3]:
X, y = df.iloc[:, :-1], df.iloc[:, -1].values
X

,0,1,2,3,4,5,6,7,8
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22
...,...,...,...,...,...,...,...,...,...
1479,YUR1_YEAST,0.81,0.62,0.43,0.17,0.5,0.0,0.53,0.22
1480,ZIP1_YEAST,0.47,0.43,0.61,0.40,0.5,0.0,0.48,0.47
1481,ZNRP_YEAST,0.67,0.57,0.36,0.19,0.5,0.0,0.56,0.22
1482,ZUO1_YEAST,0.43,0.40,0.60,0.16,0.5,0.0,0.53,0.39


In [4]:
# Transform labels to numerical values
le = LabelEncoder()
y = le.fit_transform(y)

In [5]:
hash_encoder = ce.HashingEncoder()
X = hash_encoder.fit_transform(X)
X = X.values
X

array([[0.  , 0.  , 1.  , ..., 0.  , 0.48, 0.22],
       [0.  , 0.  , 0.  , ..., 0.  , 0.53, 0.22],
       [0.  , 0.  , 0.  , ..., 0.  , 0.53, 0.22],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.56, 0.22],
       [0.  , 0.  , 0.  , ..., 0.  , 0.53, 0.39],
       [0.  , 1.  , 0.  , ..., 0.  , 0.53, 0.22]])

In [6]:
num_dimensions = X.shape[1]
num_classes = len(np.unique(y))

print("Num dimensions: ", num_dimensions)
print("Num classes: ", num_classes)

Num dimensions:  16
Num classes:  10


In [7]:
encoding = np.eye(num_classes)
encoding

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [8]:
# Define models
models = [
    ('Baseline', SVMRandomInvariantsECOC(encoding)),
    ('Vapnik', SVMRandomInvariantsECOC(encoding, invariant_type=InvariantTypes.VAPNIK, tolerance=20)),
    ('Projections', SVMRandomInvariantsECOC(encoding, invariant_type=InvariantTypes.PROJECTION, tolerance=20)),
    ('Hyperplanes', SVMRandomInvariantsECOC(encoding, invariant_type=InvariantTypes.HYPERPLANE, tolerance=20)),
]

In [9]:
# Define models parameters
models_parameters = [
    {
        'kernel': ['rbf'],
        'C': [0.001, 0.01, 0.1, 1.0, 10.0],
        'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
        'delta': [0.001, 0.01, 0.1, 1.0],
        'num_invariants': [0],
    },
    {
        'kernel': ['rbf'],
        'C': [0.001, 0.01, 0.1, 1.0, 10.0],
        'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
        'delta': [0.001, 0.01, 0.1, 1.0],
        'num_invariants': np.arange(1, num_dimensions + 2),
    },
    {
        'kernel': ['rbf'],
        'C': [0.001, 0.01, 0.1, 1.0, 10.0],
        'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
        'delta': [0.001, 0.01, 0.1, 1.0],
        'num_invariants': np.arange(1, num_dimensions + 1),
    },
    {
        'kernel': ['rbf'],
        'C': [0.001, 0.01, 0.1, 1.0, 10.0],
        'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
        'delta': [0.001, 0.01, 0.1, 1.0],
        'num_invariants': np.arange(1, num_dimensions + 1),
    }
]

In [10]:
# Define seeds and train sizes
seeds = [47, 4, 81]
train_sizes = [1.0, 0.5, 0.1]

In [11]:
results_df = run_experiment(X, y, seeds, train_sizes, models, models_parameters)
results_df.to_csv('results/yeast.csv', index=False)

-------------------- EXPERIMENT 1 --------------------




------ Using 1.0 of the training data ------


Training Baseline model

Running experiment with random_state=47
Best estimator:  SVMRandomInvariantsECOC(C=0.001,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=0.1, num_invariants=0, random_state=47)
Accuracy:  0.5286195286195287
Running experiment with random_state=4
Best estimator:  SVMRandomInvariantsECOC(C=0.001,
                        encoding=array([[1., 0., 0., 0., 0., 0., 

Best estimator:  SVMRandomInvariantsECOC(C=0.01, delta=1.0,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=1.0,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=3, random_state=81, tolerance=20)
Accuracy:  0.5185185185185185





------ Using 0.5 of the training data ------


Training Baseline model

Running experiment with random_state=47


/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.001,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=0.1, num_invariants=0, random_state=47)
Accuracy:  0.5286195286195287
Running experiment with random_state=4


/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


...

Best estimator:  SVMRandomInvariantsECOC(C=0.1, delta=1.0,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=0.1,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=13, random_state=47, tolerance=20)
Accuracy:  0.46464646464646464
Running experiment with random_state=4


/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.01, delta=1.0,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=0.1,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=2, random_state=4, tolerance=20)
Accuracy:  0.4276094276094276
Running experiment with random_state=81


/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.01, delta=1.0,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=0.1,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=12, random_state=81, tolerance=20)
Accuracy:  0.2828282828282828



